In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

from scipy import stats
from sklearn.metrics import mean_squared_error


'''
Author: Enrico Ceccolini
    TODO
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
#end_time = pd.to_datetime('2014-11-01')

infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_short_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_short_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))

### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(long_jobs, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 35284 long jobs in the Andrea's interval
after the clean, jobs_to_nodes_whole_data contains 444610 records
merged_jobs_to_nodes contains 41216 records


In [3]:
long_jobs['real_pow'].mean()

110.35204538702968

In [6]:
long_jobs['pred_overAll'].mean()

119.68048664618362

In [7]:
long_jobs[long_jobs['real_pow_quality']>0.5]['real_pow'].mean()

115.31058950116889

In [16]:
long_jobs[long_jobs['real_pow_quality']>0.5]['pred_userJob'].mean()

111.69441892099296

In [4]:
long_jobs['real_pow'].max()

4839.632715117781

In [5]:
long_jobs['real_pow'].min()

0.0

### assing zero quality to jobs with negative consumption

In [10]:
long_jobs[long_jobs['real_pow'] <= 3.4]['real_pow_quality']

job_id
30       0.997797
80       1.000000
605      1.000000
752      0.991667
755      0.994536
766      1.000000
912      0.871883
915      0.997268
1025     0.994550
1073     0.994536
1270     1.000000
1893     0.100000
4283     1.000000
4287     1.000000
4315     1.000000
4316     1.000000
4463     1.000000
6097     0.801002
6177     1.000000
6178     0.516062
6289     1.000000
6294     1.000000
6295     1.000000
6326     1.000000
6345     1.000000
6346     1.000000
6348     1.000000
6349     0.624746
6383     1.000000
6633     0.627266
           ...   
90172    1.000000
90173    1.000000
90179    1.000000
90180    1.000000
90181    1.000000
90192    1.000000
90194    1.000000
90207    1.000000
90208    1.000000
90221    1.000000
90222    1.000000
90223    1.000000
90236    1.000000
90237    1.000000
90252    1.000000
90284    1.000000
90285    1.000000
90286    1.000000
90299    1.000000
90321    1.000000
90322    1.000000
90338    1.000000
90339    1.000000
90340    1.000000
903

In [13]:
long_jobs.at[long_jobs['real_pow'] <= 3.4, 'real_pow_quality'] = 0.0

In [16]:
long_jobs.at[long_jobs['real_pow'] <= 0.0, 'real_pow'] = 0.0

In [17]:
long_jobs.to_csv(infile_long_jobs + ".csv")

$$RMSE = \sqrt{(\sum_{i=1}^{N}{\frac{(y_i-f_i)^2}{N}})}$$

In [39]:
long_jobs['variance_perNode'] = (long_jobs['real_pow'] - long_jobs['pred_perNode'])**2 / long_jobs['variance_perNode'].shape[0]

In [40]:
math.sqrt(long_jobs['variance_perNode'].sum())

141.55374774830088

In [41]:
math.sqrt(long_jobs['variance_perNode'].sum()) / long_jobs['real_pow'].mean()

1.2827469327989323

In [42]:
long_jobs['real_pow'].max() - long_jobs['real_pow'].min()

4839.632715117781

In [43]:
math.sqrt(long_jobs['variance_perNode'].sum()) / (long_jobs['real_pow'].max() - long_jobs['real_pow'].min())

0.02924886165558039

In [ ]:
# NMSE

In [32]:
long_jobs['variance_perNode'] = ((long_jobs['real_pow'] - long_jobs['pred_perNode'])**2) / (long_jobs['real_pow'].mean()*long_jobs['pred_perNode'].mean())

In [33]:
long_jobs['variance_perNode'].sum() / long_jobs['variance_perNode'].shape[0]

1.8467945697359052

In [34]:
long_jobs['variance_perNode'] = ((long_jobs['real_pow'] - long_jobs['pred_perNode'])**2) / (long_jobs['real_pow'].max()-long_jobs['real_pow'].min())
long_jobs['variance_perNode'].sum() / long_jobs['variance_perNode'].shape[0]

4.140285984718976